In [1]:
# !pip install pyspark

In [2]:
# !pip install --upgrade pip

In [3]:
# Import SparkSession
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode,col,lit

# Create SparkSession 
spark = SparkSession.builder \
      .master("local[1]") \
      .appName("SparkByExamples.com") \
      .getOrCreate()

In [4]:
dataList = [("Java", 20000), ("Python", 100000), ("Scala", 3000)]
rdd=spark.sparkContext.parallelize(dataList)

In [5]:
data = [('James','','Smith','1991-04-01','M',3000),
  ('Michael','Rose','','2000-05-19','M',4000),
  ('Robert','','Williams','1978-09-05','M',4000),
  ('Maria','Anne','Jones','1967-12-01','F',4000),
  ('Jen','Mary','Brown','1980-02-17','F',-1)
]

columns = ["firstname","middlename","lastname","dob","gender","salary"]
df = spark.createDataFrame(data=data, schema = columns)

In [6]:
df.show()

+---------+----------+--------+----------+------+------+
|firstname|middlename|lastname|       dob|gender|salary|
+---------+----------+--------+----------+------+------+
|    James|          |   Smith|1991-04-01|     M|  3000|
|  Michael|      Rose|        |2000-05-19|     M|  4000|
|   Robert|          |Williams|1978-09-05|     M|  4000|
|    Maria|      Anne|   Jones|1967-12-01|     F|  4000|
|      Jen|      Mary|   Brown|1980-02-17|     F|    -1|
+---------+----------+--------+----------+------+------+



In [7]:
import requests, json
def read_api():
    url = "https://bloomberg-market-and-financial-news.p.rapidapi.com/news/list"
    querystring = {"id":"markets"}
    headers = {
        "X-RapidAPI-Key": "15e779e164msha744913e50e0e28p1ebf46jsnd545e128dbd9",
        "X-RapidAPI-Host": "bloomberg-market-and-financial-news.p.rapidapi.com"
    }
    normalized_data = dict()
    data = requests.get(url, headers=headers, params=querystring).json() 
    normalized_data["_data"] = data # Normalize payload to handle array situtations
    return json.dumps(normalized_data)

In [8]:
import requests, json
def read_api_2():
    url = "https://jsonplaceholder.typicode.com/posts"
    data = requests.get(url).json() 
    return json.dumps(data)

In [9]:
# Read data into Data Frame
# Create payload rdd
payload = read_api_2()
with open("response.json", "w") as outfile:
    outfile.write(payload)

In [10]:
df = spark.read.json("response.json")
df.printSchema()
df.show()

root
 |-- body: string (nullable = true)
 |-- id: long (nullable = true)
 |-- title: string (nullable = true)
 |-- userId: long (nullable = true)

+--------------------+---+--------------------+------+
|                body| id|               title|userId|
+--------------------+---+--------------------+------+
|quia et suscipit\...|  1|sunt aut facere r...|     1|
|est rerum tempore...|  2|        qui est esse|     1|
|et iusto sed quo ...|  3|ea molestias quas...|     1|
|ullam et saepe re...|  4|eum et est occaecati|     1|
|repudiandae venia...|  5|  nesciunt quas odio|     1|
|ut aspernatur cor...|  6|dolorem eum magni...|     1|
|dolore placeat qu...|  7|magnam facilis autem|     1|
|dignissimos aperi...|  8|dolorem dolore es...|     1|
|consectetur animi...|  9|nesciunt iure omn...|     1|
|quo et expedita m...| 10|optio molestias i...|     1|
|delectus reiciend...| 11|et ea vero quia l...|     2|
|itaque id aut mag...| 12|in quibusdam temp...|     2|
|aut dicta possimu...| 13|do

In [11]:
print((df.count(), len(df.columns)))

(100, 4)


In [12]:
df.printSchema()

root
 |-- body: string (nullable = true)
 |-- id: long (nullable = true)
 |-- title: string (nullable = true)
 |-- userId: long (nullable = true)



In [13]:
df.head(5)

[Row(body='quia et suscipit\nsuscipit recusandae consequuntur expedita et cum\nreprehenderit molestiae ut ut quas totam\nnostrum rerum est autem sunt rem eveniet architecto', id=1, title='sunt aut facere repellat provident occaecati excepturi optio reprehenderit', userId=1),
 Row(body='est rerum tempore vitae\nsequi sint nihil reprehenderit dolor beatae ea dolores neque\nfugiat blanditiis voluptate porro vel nihil molestiae ut reiciendis\nqui aperiam non debitis possimus qui neque nisi nulla', id=2, title='qui est esse', userId=1),
 Row(body='et iusto sed quo iure\nvoluptatem occaecati omnis eligendi aut ad\nvoluptatem doloribus vel accusantium quis pariatur\nmolestiae porro eius odio et labore et velit aut', id=3, title='ea molestias quasi exercitationem repellat qui ipsa sit aut', userId=1),
 Row(body='ullam et saepe reiciendis voluptatem adipisci\nsit amet autem assumenda provident rerum culpa\nquis hic commodi nesciunt rem tenetur doloremque ipsam iure\nquis sunt voluptatem rerum i

In [14]:
df.select('userid').distinct().show(100)

+------+
|userid|
+------+
|     7|
|     6|
|     9|
|     5|
|     1|
|    10|
|     3|
|     8|
|     2|
|     4|
+------+



In [15]:
df.select('body').distinct().count()

100

In [16]:
df.crosstab('id', 'userid').show(10)

+---------+---+---+---+---+---+---+---+---+---+---+
|id_userid|  1| 10|  2|  3|  4|  5|  6|  7|  8|  9|
+---------+---+---+---+---+---+---+---+---+---+---+
|        7|  1|  0|  0|  0|  0|  0|  0|  0|  0|  0|
|       51|  0|  0|  0|  0|  0|  0|  1|  0|  0|  0|
|       15|  0|  0|  1|  0|  0|  0|  0|  0|  0|  0|
|       54|  0|  0|  0|  0|  0|  0|  1|  0|  0|  0|
|       11|  0|  0|  1|  0|  0|  0|  0|  0|  0|  0|
|       69|  0|  0|  0|  0|  0|  0|  0|  1|  0|  0|
|       29|  0|  0|  0|  1|  0|  0|  0|  0|  0|  0|
|       42|  0|  0|  0|  0|  0|  1|  0|  0|  0|  0|
|       73|  0|  0|  0|  0|  0|  0|  0|  0|  1|  0|
|       87|  0|  0|  0|  0|  0|  0|  0|  0|  0|  1|
+---------+---+---+---+---+---+---+---+---+---+---+
only showing top 10 rows



In [17]:
df.filter(df.userId > 5).count()

50

In [18]:
df.columns

['body', 'id', 'title', 'userId']

# Countries task

In [19]:
from pyspark.sql.types import ArrayType, StringType
from pyspark.sql import functions as F
import requests
# Make a request to get information about: "Bulgaria", "Germany", "Italy", "Peru"
country_name = "Bulgaria"
response = requests.get(f"https://restcountries.com/v3.1/name/{country_name}")
# Parse the response as JSON
country_data = response.json()
country_data
# name , region, capital, currency, border_country_full_name

[{'name': {'common': 'Bulgaria',
   'official': 'Republic of Bulgaria',
   'nativeName': {'bul': {'official': 'Република България',
     'common': 'България'}}},
  'tld': ['.bg'],
  'cca2': 'BG',
  'ccn3': '100',
  'cca3': 'BGR',
  'cioc': 'BUL',
  'independent': True,
  'status': 'officially-assigned',
  'unMember': True,
  'currencies': {'BGN': {'name': 'Bulgarian lev', 'symbol': 'лв'}},
  'idd': {'root': '+3', 'suffixes': ['59']},
  'capital': ['Sofia'],
  'altSpellings': ['BG', 'Republic of Bulgaria', 'Република България'],
  'region': 'Europe',
  'subregion': 'Southeast Europe',
  'languages': {'bul': 'Bulgarian'},
  'translations': {'ara': {'official': 'جمهورية بلغاريا', 'common': 'بلغاريا'},
   'bre': {'official': 'Republik Bulgaria', 'common': 'Bulgaria'},
   'ces': {'official': 'Bulharská republika', 'common': 'Bulharsko'},
   'cym': {'official': 'Gweriniaeth Bwlgaria', 'common': 'Bwlgaria'},
   'deu': {'official': 'Republik Bulgarien', 'common': 'Bulgarien'},
   'est': {'offi

In [20]:
# json_rdd = spark.sparkContext.parallelize([json.dumps(country_data)])
# df = spark.read.json(json_rdd)
df = spark.createDataFrame(country_data)
# df.printSchema()

In [21]:
# name , region, capital, currency, border_country_full_name
df.select("name.common").show(truncate=False)

+--------+
|common  |
+--------+
|Bulgaria|
+--------+



In [22]:
df_currencies = df.select(explode(col("currencies")))
df_currencies = df_currencies.select(explode(col("value")))
df_currencies.show()
# currency_name = df_currencies.first()['value']
# currency_name

+------+-------------+
|   key|        value|
+------+-------------+
|  name|Bulgarian lev|
|symbol|           лв|
+------+-------------+



In [23]:
df_currencies.show()

+------+-------------+
|   key|        value|
+------+-------------+
|  name|Bulgarian lev|
|symbol|           лв|
+------+-------------+



In [24]:
import pyspark.sql.functions as f
def getCountryName(country):
    print(country)
    response = requests.get(f"https://restcountries.com/v3.1/alpha/{country}")
    # Parse the response as JSON
    data = response.json()
    print(data)
    return data[0]['name']['common'];

In [25]:
from pyspark.sql.functions import explode
df = df.select("name.common","region","capital","borders")
df.show()

+--------+------+-------+--------------------+
|  common|region|capital|             borders|
+--------+------+-------+--------------------+
|Bulgaria|Europe|[Sofia]|[GRC, MKD, ROU, S...|
+--------+------+-------+--------------------+



In [26]:
name = getCountryName("BUL")
type(name)

BUL
[{'name': {'common': 'Bulgaria', 'official': 'Republic of Bulgaria', 'nativeName': {'bul': {'official': 'Република България', 'common': 'България'}}}, 'tld': ['.bg'], 'cca2': 'BG', 'ccn3': '100', 'cca3': 'BGR', 'cioc': 'BUL', 'independent': True, 'status': 'officially-assigned', 'unMember': True, 'currencies': {'BGN': {'name': 'Bulgarian lev', 'symbol': 'лв'}}, 'idd': {'root': '+3', 'suffixes': ['59']}, 'capital': ['Sofia'], 'altSpellings': ['BG', 'Republic of Bulgaria', 'Република България'], 'region': 'Europe', 'subregion': 'Southeast Europe', 'languages': {'bul': 'Bulgarian'}, 'translations': {'ara': {'official': 'جمهورية بلغاريا', 'common': 'بلغاريا'}, 'bre': {'official': 'Republik Bulgaria', 'common': 'Bulgaria'}, 'ces': {'official': 'Bulharská republika', 'common': 'Bulharsko'}, 'cym': {'official': 'Gweriniaeth Bwlgaria', 'common': 'Bwlgaria'}, 'deu': {'official': 'Republik Bulgarien', 'common': 'Bulgarien'}, 'est': {'official': 'Bulgaaria Vabariik', 'common': 'Bulgaaria'}, '

str

In [27]:
df.show()

+--------+------+-------+--------------------+
|  common|region|capital|             borders|
+--------+------+-------+--------------------+
|Bulgaria|Europe|[Sofia]|[GRC, MKD, ROU, S...|
+--------+------+-------+--------------------+



# Apply transformations on boarders and capital columns

In [28]:
df = df.withColumn("borders" , explode("borders"))
df = df.withColumn("capital" , explode("capital"))
custom_getCountryName = F.udf(getCountryName, StringType())
df = df.withColumn('borders', custom_getCountryName(df.borders))

In [29]:
df.show()

+--------+------+-------+---------------+
|  common|region|capital|        borders|
+--------+------+-------+---------------+
|Bulgaria|Europe|  Sofia|         Greece|
|Bulgaria|Europe|  Sofia|North Macedonia|
|Bulgaria|Europe|  Sofia|        Romania|
|Bulgaria|Europe|  Sofia|         Serbia|
|Bulgaria|Europe|  Sofia|         Turkey|
+--------+------+-------+---------------+



In [30]:
df = df.groupby("common","region","capital").agg(F.collect_set("borders").alias("borders"))
# df=df.withColumnRenamed("collect_set(borders)","borders")
df.show(truncate=False)

+--------+------+-------+--------------------------------------------------+
|common  |region|capital|borders                                           |
+--------+------+-------+--------------------------------------------------+
|Bulgaria|Europe|Sofia  |[North Macedonia, Serbia, Greece, Turkey, Romania]|
+--------+------+-------+--------------------------------------------------+



# Make the code reusable

In [36]:
from pyspark.sql.functions import explode
# Make a request to get information about: "Bulgaria", "Germany", "Italy", "Peru"
# Parse the response as JSON
country_data

def getCountryData(country):
    response = requests.get(f"https://restcountries.com/v3.1/name/{country}")
    return response.json();


def getCountryName(country):
    print(country)
    response = requests.get(f"https://restcountries.com/v3.1/alpha/{country}")
    # Parse the response as JSON
    data = response.json()
    print(data)
    return data[0]['name']['common'];

def extractCurrencyName(df):
    df = df.select(explode(col("currencies")))
    df = df.select(explode(col("value")))
    return df.first()['value']

def prepareDataFrame(country_data):
    df = spark.createDataFrame(country_data)
    currency_name = extractCurrencyName(df)
    df = df.select(col("name.common").alias("name"),"region","capital","borders")
    df = df.withColumn("borders" , explode("borders"))
    df = df.withColumn("capital" , explode("capital"))
    df = df.withColumn("currency" , lit(currency_name))
    custom_getCountryName = F.udf(getCountryName, StringType())
    df = df.withColumn('borders', custom_getCountryName(df.borders))
    return df.groupby("name","region","capital","currency").agg(F.collect_set("borders").alias("borders"));

def showCountryData(country_list):
    for country in country_list:
        print(country)
        country_data = getCountryData(country)
        country_df = prepareDataFrame(country_data)
        country_df.show(truncate=False)


country_list = ["Bulgaria", "Germany", "Italy", "Peru"]
showCountryData(country_list)

Bulgaria
+--------+------+-------+-------------+--------------------------------------------------+
|name    |region|capital|currency     |borders                                           |
+--------+------+-------+-------------+--------------------------------------------------+
|Bulgaria|Europe|Sofia  |Bulgarian lev|[North Macedonia, Serbia, Greece, Turkey, Romania]|
+--------+------+-------+-------------+--------------------------------------------------+

Germany
+-------+------+-------+--------+------------------------------------------------------------------------------------------+
|name   |region|capital|currency|borders                                                                                   |
+-------+------+-------+--------+------------------------------------------------------------------------------------------+
|Germany|Europe|Berlin |Euro    |[France, Austria, Poland, Denmark, Switzerland, Luxembourg, Netherlands, Czechia, Belgium]|
+-------+------+-------+---

In [ ]:
df.show()